<a href="https://colab.research.google.com/github/inoue0426/scRNA-Data-Imputation-comparison/blob/main/notebooks/scImpute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages("devtools")
library(devtools)

install_github("Vivianstats/scImpute")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Skipping install of 'scImpute' from a github remote, the SHA1 (78556ee2) has not changed since last install.
  Use `force = TRUE` to force installation



In [ ]:
url <- 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE57249&format=file&file=GSE57249%5Ffpkm%2Etxt%2Egz'
tmp <- tempfile()
##
download.file(url,tmp)
##
data <- read.csv(
  gzfile(tmp),
  sep="\t",
  header=TRUE,
  stringsAsFactors=FALSE)

In [ ]:
head(data)

,ID,GSM1377859,GSM1377860,GSM1377861,GSM1377862,GSM1377863,GSM1377864,GSM1377865,GSM1377866,GSM1377867,⋯,GSM1377905,GSM1377906,GSM1377907,GSM1377908,GSM1377909,GSM1377910,GSM1377911,GSM1377912,GSM1377913,GSM1377914
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSMUSG00000000001,25.8078,36.75610,8.87692,24.5712,31.22550,24.7722,40.745800,38.41550,43.4644,⋯,28.04380,5.55538,16.9205,14.552100,51.47380,1.285560,40.56320,0.00000,0.084052,0.000000
2,ENSMUSG00000000028,93.4291,92.11650,94.59080,107.0380,121.44900,139.3230,0.960904,162.72700,121.1760,⋯,112.88000,27.00820,105.0870,70.179700,41.61460,131.129000,56.93390,8.76009,3.831900,2.785230
3,ENSMUSG00000000031,0.0000,0.00000,0.00000,0.0000,0.00000,0.0000,0.000000,0.00000,0.0000,⋯,0.00000,0.00000,0.0000,0.261558,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000
4,ENSMUSG00000000037,37.9544,22.43050,23.34200,42.2728,23.85790,32.4989,15.721000,37.73550,48.2360,⋯,1.08452,0.00000,0.0000,0.000000,4.74386,0.000000,2.09974,0.00000,0.000000,0.000000
5,ENSMUSG00000000049,0.0000,0.00000,0.00000,0.0000,0.00000,0.0000,0.000000,1.19912,0.0000,⋯,0.00000,0.00000,0.0000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000
6,ENSMUSG00000000056,11.7348,5.61334,7.52622,6.2115,6.20216,28.3024,39.470500,37.35680,11.8725,⋯,0.00000,0.00000,0.0000,0.000000,3.39413,0.035212,9.44850,0.00000,0.000000,0.108556


In [ ]:
data = subset(data, select = -c(ID))
head(data)

,GSM1377859,GSM1377860,GSM1377861,GSM1377862,GSM1377863,GSM1377864,GSM1377865,GSM1377866,GSM1377867,GSM1377868,⋯,GSM1377905,GSM1377906,GSM1377907,GSM1377908,GSM1377909,GSM1377910,GSM1377911,GSM1377912,GSM1377913,GSM1377914
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,25.8078,36.75610,8.87692,24.5712,31.22550,24.7722,40.745800,38.41550,43.4644,32.3039,⋯,28.04380,5.55538,16.9205,14.552100,51.47380,1.285560,40.56320,0.00000,0.084052,0.000000
2,93.4291,92.11650,94.59080,107.0380,121.44900,139.3230,0.960904,162.72700,121.1760,132.2440,⋯,112.88000,27.00820,105.0870,70.179700,41.61460,131.129000,56.93390,8.76009,3.831900,2.785230
3,0.0000,0.00000,0.00000,0.0000,0.00000,0.0000,0.000000,0.00000,0.0000,0.0000,⋯,0.00000,0.00000,0.0000,0.261558,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000
4,37.9544,22.43050,23.34200,42.2728,23.85790,32.4989,15.721000,37.73550,48.2360,34.3155,⋯,1.08452,0.00000,0.0000,0.000000,4.74386,0.000000,2.09974,0.00000,0.000000,0.000000
5,0.0000,0.00000,0.00000,0.0000,0.00000,0.0000,0.000000,1.19912,0.0000,0.0000,⋯,0.00000,0.00000,0.0000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000
6,11.7348,5.61334,7.52622,6.2115,6.20216,28.3024,39.470500,37.35680,11.8725,27.3852,⋯,0.00000,0.00000,0.0000,0.000000,3.39413,0.035212,9.44850,0.00000,0.000000,0.108556


In [ ]:
scimpute(# full path to raw count matrix
         count_path = gzfile(tmp), 
         infile = "csv",           # format of input file
         outfile = "csv",          # format of output file
         out_dir = "./",           # full path to output directory
         labeled = FALSE,          # cell type labels not available
         drop_thre = 0.5,          # threshold set on dropout probability
         Kcluster = 2,             # 2 cell subpopulations
         ncores = 10)              # number of cores used in parallel computation

ERROR: ignored